In [3]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [4]:
# read in all the words
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [5]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [6]:
C = torch.randn((27, 2))
C.shape

torch.Size([27, 2])

In [7]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?


X, Y = [], []
for w in words[:5]:

  #print(w)
  context = [0] * block_size
  # print(context)
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    # print(X)
    
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
    # print(context)
print(X)
X = torch.tensor(X)
Y = torch.tensor(Y)
print(X.shape, Y.shape)


... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
... ---> a
..a ---> v
.av ---> a
ava ---> .
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .
[[0, 0, 0], [0, 0, 5], [0, 5, 13], [5, 13, 13], [13, 13, 1], [0, 0, 0], [0, 0, 15], [0, 15, 12], [15, 12, 9], [12, 9, 22], [9, 22, 9], [22, 9, 1], [0, 0, 0], [0, 0, 1], [0, 1, 22], [1, 22, 1], [0, 0, 0], [0, 0, 9], [0, 9, 19], [9, 19, 1], [19, 1, 2], [1, 2, 5], [2, 5, 12], [5, 12, 12], [12, 12, 1], [0, 0, 0], [0, 0, 19], [0, 19, 15], [19, 15, 16], [15, 16, 8], [16, 8, 9], [8, 9, 1]]
torch.Size([32, 3]) torch.Size([32])


In [8]:
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [9]:
X.shape

torch.Size([32, 3])

In [10]:
context

[9, 1, 0]

In [11]:
C[X]


tensor([[[ 0.3838, -1.1150],
         [ 0.3838, -1.1150],
         [ 0.3838, -1.1150]],

        [[ 0.3838, -1.1150],
         [ 0.3838, -1.1150],
         [-0.2136,  0.1180]],

        [[ 0.3838, -1.1150],
         [-0.2136,  0.1180],
         [ 0.6651,  0.2696]],

        [[-0.2136,  0.1180],
         [ 0.6651,  0.2696],
         [ 0.6651,  0.2696]],

        [[ 0.6651,  0.2696],
         [ 0.6651,  0.2696],
         [-1.5927, -1.7184]],

        [[ 0.3838, -1.1150],
         [ 0.3838, -1.1150],
         [ 0.3838, -1.1150]],

        [[ 0.3838, -1.1150],
         [ 0.3838, -1.1150],
         [ 1.4631, -1.7455]],

        [[ 0.3838, -1.1150],
         [ 1.4631, -1.7455],
         [ 0.1884, -0.2648]],

        [[ 1.4631, -1.7455],
         [ 0.1884, -0.2648],
         [ 0.3091, -0.6728]],

        [[ 0.1884, -0.2648],
         [ 0.3091, -0.6728],
         [ 0.5681, -0.4695]],

        [[ 0.3091, -0.6728],
         [ 0.5681, -0.4695],
         [ 0.3091, -0.6728]],

        [[ 0.5681, -0

In [12]:
X.shape


torch.Size([32, 3])

In [13]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [14]:

torch.cat([emb[:, 0, :],emb[:, 1, :],emb[:, 2, :]],1) # select 2nd dimensions 3 columns explicitly and changes int to 2d

tensor([[ 0.3838, -1.1150,  0.3838, -1.1150,  0.3838, -1.1150],
        [ 0.3838, -1.1150,  0.3838, -1.1150, -0.2136,  0.1180],
        [ 0.3838, -1.1150, -0.2136,  0.1180,  0.6651,  0.2696],
        [-0.2136,  0.1180,  0.6651,  0.2696,  0.6651,  0.2696],
        [ 0.6651,  0.2696,  0.6651,  0.2696, -1.5927, -1.7184],
        [ 0.3838, -1.1150,  0.3838, -1.1150,  0.3838, -1.1150],
        [ 0.3838, -1.1150,  0.3838, -1.1150,  1.4631, -1.7455],
        [ 0.3838, -1.1150,  1.4631, -1.7455,  0.1884, -0.2648],
        [ 1.4631, -1.7455,  0.1884, -0.2648,  0.3091, -0.6728],
        [ 0.1884, -0.2648,  0.3091, -0.6728,  0.5681, -0.4695],
        [ 0.3091, -0.6728,  0.5681, -0.4695,  0.3091, -0.6728],
        [ 0.5681, -0.4695,  0.3091, -0.6728, -1.5927, -1.7184],
        [ 0.3838, -1.1150,  0.3838, -1.1150,  0.3838, -1.1150],
        [ 0.3838, -1.1150,  0.3838, -1.1150, -1.5927, -1.7184],
        [ 0.3838, -1.1150, -1.5927, -1.7184,  0.5681, -0.4695],
        [-1.5927, -1.7184,  0.5681, -0.4

In [15]:
torch.cat([emb[:, 0, :],emb[:, 1, :],emb[:, 2, :]],1).shape #[32,3,2] = 32,6

torch.Size([32, 6])

In [16]:
#without explicitly stating
torch.cat(torch.unbind(emb , 1), 1).shape #select 2nd D and convert

torch.Size([32, 6])

In [17]:
a = torch.arange(18)
a

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

In [18]:
a.view(3,3,2)

tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5]],

        [[ 6,  7],
         [ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15],
         [16, 17]]])

In [19]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [20]:
sum(p.nelement() for p in parameters) # number of parameters in total

11897

In [21]:
for p in parameters:
  p.requires_grad = True

In [ ]:
#to find the best learning rate

lre = torch.linspace(-3, 0, 1000)#-3,0 are exponential 10**-3 = .001
lrs = 10**lre